# Availability


In [1]:
import pandas as pd
import numpy as np
from datetime import date

import platform
import pyodbc

from sqlalchemy.engine import URL
from sqlalchemy import create_engine, text

import matplotlib.pyplot as plt
import dtale
import sweetviz
# import pandas_profiling

import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import train_test_split
from scipy.interpolate import UnivariateSpline

pd.set_option('display.max.columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [25]:
# def print_full(x):
#     pd.set_option('display.max_rows', None)
#     pd.set_option('display.max_columns', None)
#     pd.set_option('display.width', 2000)
#     pd.set_option('display.float_format', '{:20,.2f}'.format)
#     pd.set_option('display.max_colwidth', None)
#     print(x)
#     pd.reset_option('display.max_rows')
#     pd.reset_option('display.max_columns')
#     pd.reset_option('display.width')
#     pd.reset_option('display.float_format')
#     pd.reset_option('display.max_colwidth')


In [2]:
import os

# Change the current working directory
os.chdir('C:/Users/DeepankarSrigyan/OneDrive - Sykes Cottages Ltd/Desktop/Project/RGA Tools/availability/')

In [3]:
def fetch_SQLserver_data(sql_code,conn_string):
    """
    Uses pyodbc.connect and pandas.read_sql to return query results as dataframe.
    :param sql_code: string, query to be submitted
    :param conn_string: string, full connection details as string
    """
    connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_string})
    engine = create_engine(connection_url)
    with engine.connect() as conn:
        df = pd.read_sql(sql=text(sql_code), con=conn)
        conn.commit()
    print('Data Imported')
    return df

if platform.system() == 'Windows':
    conn_CDM = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-04;DATABASE=CDM;trusted_connection=Yes'
    conn_l0 = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-04;DATABASE=Sykes_DW_Level0_Staging;trusted_connection=Yes'
    conn_l2 = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-04;DATABASE=Sykes_DW_Level2_ADM;trusted_connection=Yes'
    conn_BOX = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-04;DATABASE=Box;trusted_connection=Yes'

    print ("Connection Completed")

Connection Completed


In [4]:
print("hello")

hello


In [ ]:
SELECT PROPERTYID
    , PropertyReference
    , PropertyLiveDate
    , DateToldWithdrawingLocal
    , AvailableFromDate
    , AvailableToDate
INTO #PROPERTY
FROM [cdm].[propertymanagement].[Property] P
WHERE p.PropertyLiveDate IS NOT NULL
    AND p.PropertyLiveDate >= '2022-09-01'
    AND (
        p.DateToldWithdrawingLocal > p.PropertyLiveDate
        OR p.DateToldWithdrawingLocal IS NULL
        )

In [ ]:
SELECT
    p.PropertyID,
    p.PropertyReference,
    p.PropertyLiveDate,
    p.DateToldWithdrawingLocal,
    p.AvailableFromDate,
    p.AvailableToDate,

    SUM(CASE WHEN a.PropertyAvailabilityStatus = 'Available' THEN 1 ELSE 0 END) AS Available_Days,
    SUM(CASE WHEN a.PropertyAvailabilityStatus = 'Not Available' THEN 1 ELSE 0 END) AS Not_Available_Days,
    COUNT(1) AS Total_Days
    INTO #AVAILABILITY
FROM
   #PROPERTY p
    LEFT JOIN cdm.propertymanagement.PropertyAvailability a
    ON p.PropertyID = a.PropertyID
WHERE 
            -- p.PropertyReference IN (2074, 20319, 906631, 913976, 913218, 1120258) AND 

     a.PropertyAvailabilityDate >= p.PropertyLiveDate
    -- AND a.PropertyAvailabilityDate <= LEAST(DATEADD(DAY, 364, p.PropertyLiveDate), GETDATE(), p.AvailableToDate)
    AND a.PropertyAvailabilityDate <= DATEADD(DAY, 364, p.PropertyLiveDate)
GROUP BY 
        p.PropertyID,
        p.PropertyReference,
        p.PropertyLiveDate,
        p.AvailableFromDate,
        p.AvailableToDate,
        p.DateToldWithdrawingLocal

In [3]:
availability=pd.read_excel('availability.xlsx','sheet1')
availability

,PropertyID,PropertyReference,PropertyLiveDate,DateToldWithdrawingLocal,Difference,churn1year,AvailableFromDate,AvailableToDate,Available_Days,Not_Available_Days,Total_Days
0,223990,1128768,2023-02-28,NaT,0,NaN,2023-03-03,NaT,362,3,365
1,225256,1129878,2023-08-25,NaT,0,NaN,2023-08-25,NaT,365,0,365
2,221376,1126413,2023-02-10,2023-10-23,255,1.0,2023-03-31,2023-10-30,214,151,365
3,279394,1152055,2024-06-14,NaT,0,NaN,2024-06-24,NaT,355,10,365
4,210746,1119529,2022-12-17,2023-04-13,117,1.0,2022-12-18,2023-12-29,364,1,365
...,...,...,...,...,...,...,...,...,...,...,...
8171,284049,1155772,2024-05-09,NaT,0,NaN,2024-05-10,NaT,364,1,365
8172,277099,1150231,2024-02-29,NaT,0,NaN,2024-03-01,NaT,364,1,365
8173,234957,1138337,2023-09-11,NaT,0,NaN,2023-09-11,NaT,365,0,365
8174,273044,1147631,2024-03-13,NaT,0,NaN,2024-03-13,NaT,365,0,365


In [4]:
availability['LagPeriod']=availability['AvailableFromDate']-availability['PropertyLiveDate']
availability

,PropertyID,PropertyReference,PropertyLiveDate,DateToldWithdrawingLocal,Difference,churn1year,AvailableFromDate,AvailableToDate,Available_Days,Not_Available_Days,Total_Days,LagPeriod
0,223990,1128768,2023-02-28,NaT,0,NaN,2023-03-03,NaT,362,3,365,3 days
1,225256,1129878,2023-08-25,NaT,0,NaN,2023-08-25,NaT,365,0,365,0 days
2,221376,1126413,2023-02-10,2023-10-23,255,1.0,2023-03-31,2023-10-30,214,151,365,49 days
3,279394,1152055,2024-06-14,NaT,0,NaN,2024-06-24,NaT,355,10,365,10 days
4,210746,1119529,2022-12-17,2023-04-13,117,1.0,2022-12-18,2023-12-29,364,1,365,1 days
...,...,...,...,...,...,...,...,...,...,...,...,...
8171,284049,1155772,2024-05-09,NaT,0,NaN,2024-05-10,NaT,364,1,365,1 days
8172,277099,1150231,2024-02-29,NaT,0,NaN,2024-03-01,NaT,364,1,365,1 days
8173,234957,1138337,2023-09-11,NaT,0,NaN,2023-09-11,NaT,365,0,365,0 days
8174,273044,1147631,2024-03-13,NaT,0,NaN,2024-03-13,NaT,365,0,365,0 days


In [5]:
availability.to_excel('availability_outcome.xlsx')

In [5]:
import matplotlib
matplotlib.use('Agg')
import sweetviz as sv

PropertyImage_report = sv.analyze(PropertyImage)
PropertyImage_report.show_html('PropertyImage.html')

                                             |                                             | [  0%]   00:00 ->…

Report PropertyImage.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [5]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(PropertyImage, title="Profiling Report")
# profile

In [33]:
PropertyImage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14722 entries, 0 to 14721
Columns: 103 entries, PropertyID to Churn_Year_1&2
dtypes: bool(53), datetime64[ns](7), float64(12), int64(12), object(19)
memory usage: 6.4+ MB
